In [102]:
import json
path = "/Users/yusufmoola/Desktop/Code Demo's/GenAiHackathon/backend/src/samplequiz.json"
path2 = "/Users/yusufmoola/Desktop/Code Demo's/GenAiHackathon/backend/src/sample_answers.json"

with open(path, 'r') as f:
    sample_data = json.load(f)

with open(path2, 'r') as f:
    sample_answers = json.load(f)

# Imports

In [ ]:
from langchain_ollama.llms import OllamaLLM
from langchain_openai import OpenAI
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_google_vertexai import VertexAI

from vertexai.generative_models import (
    GenerativeModel,
    HarmCategory,
    HarmBlockThreshold,
    Part,
    SafetySetting,
)


In [12]:
import google.generativeai as genai 
import os 
genai.configure(api_key= os.environ['GOOGLE_API_KEY'])

# Prompt Strengths

In [177]:
llm = OllamaLLM(
    model= "mathstral:latest",
    temperature=0.3
)


prompt_strength = '''


You are a data-driven educational assistant helping a math teacher analyze a student's performance on an advanced functions quiz. 
You are given two JSON files:

## 1. quiz_questions – This includes quiz metadata:

question_number: A unique identifier for each question
question: The question text
answer: The correct answer (symbol-limited or numeric)
difficulty: The difficulty level (easy, medium, or hard)
topic: The curriculum topic of the question

## 2. user_responses – This includes:

question_number: Matching ID from the quiz
question: Repeated for context
user_response: The student’s response

Here are the quiz questions json(1):
{}

Here are the user responses json(2):
{}

## Instructions
- First check which answers were correct by comparing student response with answer from the 2 jsons
- Then, identify topics that the student did well in based on their correct answers
- only include the questions they got correcr



## Output Format
- Structure output in neat paragraphs with headings
- DO NOT RETURN CODE
- DO NOT RETURN JSON
- DO NOT WRAP ANSWER IN (''' ''')


THIS IS THE STRUCTURE I WANT YOU TO FOLLOW WHEN ANSWERING:

"
## Students correct responses from the quiz

**Correct Responses:**

*   **Question number:**
    *   Topic: (topic)
    *   Difficulty: (difficulty)


"
'''

In [178]:
prompt_final = prompt_strength.format(sample_data, sample_answers)
res_strength = llm.invoke(prompt_final)
print(res_strength)

1. Students Correct Responses from the Quiz

**Correct Responses:**

*   **Question number:** 1
    *   Topic: Exponential Equations
    *   Difficulty: Easy
    
*   **Question number:** 3
    *   Topic: Radian Measure
    *   Difficulty: Easy
    
*   **Question number:** 4
    *   Topic: Trigonometric Identity
    *   Difficulty: Medium
    
*   **Question number:** 6
    *   Topic: Polynomial Roots
    *   Difficulty: Medium
    
*   **Question number:** 7
    *   Topic: Average Rate of Change
    *   Difficulty: Hard
    
*   **Question number:** 8
    *   Topic: Function Transformations
    *   Difficulty: Medium
    
*   **Question number:** 10
    *   Topic: Function Composition
    *   Difficulty: Hard

The student answered correctly on questions related to exponential equations, radian measure, trigonometric identities, polynomial roots, average rate of change, function transformations, and function composition.


# Prompt Weakness

In [170]:
llm = OllamaLLM(
    model= "mathstral:latest",
    temperature=0.3
)


prompt_weakness = '''

THIS IS THE STRUCTURE I WANT YOU TO FOLLOW WHEN ANSWERING:

"
Here's an analysis of the student's quiz performance:

**Incorrect Responses:**

*   **Question number:**
    *   Topic: (topic)
    *   Difficulty: (difficulty)
    *   Correct Answer: **answer
    *   Student Answer: **response


**Aggregate Insights:**

*   **Topics of Struggle:** 
*   **Difficulty Level:**  
*   **Patterns:**  

"

You are a data-driven educational assistant helping a math teacher analyze a student's performance on an advanced functions quiz. 
You are given two JSON files:

## 1. quiz_questions – This includes quiz metadata:

question_number: A unique identifier for each question
question: The question text
answer: The correct answer (symbol-limited or numeric)
difficulty: The difficulty level (easy, medium, or hard)
topic: The curriculum topic of the question

## 2. user_responses – This includes:

question_number: Matching ID from the quiz
question: Repeated for context
user_response: The student’s response

Here are the quiz questions json(1):
{}

Here are the user responses json(2):
{}

## Instructions
- Match user responses with correct answers using question_number.
- Identify incorrect responses.
- For each incorrect response, record:
- The topic and difficulty of the question
- The correct answer and the student’s incorrect answer


### Aggregate insights:
- Which topics the student struggled with the most
- Which difficulty levels are most problematic
- Any pattern (e.g., wrong on most hard questions or all questions in a topic like logarithmic equations)

## Output Format
- DO NOT RETURN CODE
- DO NOT RETURN JSON
- DO NOT WRAP ANSWER IN (''' ''')
'''

# LLM Config

In [137]:
safety_settings = {
                HarmCategory.HARM_CATEGORY_UNSPECIFIED: HarmBlockThreshold.BLOCK_ONLY_HIGH,
                HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
                HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
                HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
                HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
}
class LLM:
    def __init__(self):
        self.llm = VertexAI(
            model_name="gemini-1.5-pro-001",
            temperature=.4,
            top_p=0.8,
            top_k=40,
            max_output_tokens=4096,
            verbose=True,
            safety_settings=safety_settings  # Add the safety configuration here
            )

In [171]:
prompt_final = prompt_weakness.format(sample_data, sample_answers)
res = llm.invoke(prompt_final)
print(res)

1. **Incorrect Responses:**

*   **Question number: 2**
    *   Topic: logarithmic properties
    *   Difficulty: medium
    *   Correct Answer: 2
    *   Student Answer: 3

*   **Question number: 5**
    *   Topic: rational functions
    *   Difficulty: hard
    *   Correct Answer: x = ±3
    *   Student Answer: x = ±2

*   **Question number: 9**
    *   Topic: logarithmic equations
    *   Difficulty: hard
    *   Correct Answer: 13.8
    *   Student Answer: 12

2. **Aggregate Insights:**

*   **Topics of Struggle:** The student struggled with questions related to logarithmic properties, rational functions, and logarithmic equations.
*   **Difficulty Level:** The student found hard difficulty level questions more challenging.
*   **Patterns:** The student made errors in the topics of logarithmic properties, rational functions, and logarithmic equations, suggesting a need for more practice or review in these areas.


# RAG 

In [184]:
from langchain_ollama.llms import OllamaLLM
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("/Users/yusufmoola/Desktop/Code Demo's/GenAiHackathon/backend/Advanced-Functions.pdf")
docs = loader.load()

from langchain_ollama import OllamaEmbeddings

Ignoring wrong pointing object 41 0 (offset 0)
Ignoring wrong pointing object 46 0 (offset 0)


In [185]:
docs

[Document(metadata={'producer': 'macOS Version 15.3.2 (Build 24D81) Quartz PDFContext', 'creator': 'Preview', 'creationdate': "D:20250322022618Z00'00'", 'title': 'math1112currb-2', 'author': 'Yusuf Moola', 'moddate': "D:20250322022618Z00'00'", 'source': "/Users/yusufmoola/Desktop/Code Demo's/GenAiHackathon/backend/Advanced-Functions.pdf", 'total_pages': 14, 'page': 0, 'page_label': '1'}, page_content='Advanced Functions, \nGrade 12\nUniversity Preparation MHF4U\nThis course extends students’ experience with functions. Students will investigate the\nproperties of polynomial, rational, logarithmic, and trigonometric functions; develop\ntechniques for combining functions; broaden their understanding of rates of change; and\ndevelop facility in applying these concepts and skills. Students will also refine their use\nof the mathematical processes necessary for success in senior mathematics. This course \nis intended both for students taking the Calculus and Vectors course as a prerequisite 

In [186]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)
documents

[Document(metadata={'producer': 'macOS Version 15.3.2 (Build 24D81) Quartz PDFContext', 'creator': 'Preview', 'creationdate': "D:20250322022618Z00'00'", 'title': 'math1112currb-2', 'author': 'Yusuf Moola', 'moddate': "D:20250322022618Z00'00'", 'source': "/Users/yusufmoola/Desktop/Code Demo's/GenAiHackathon/backend/Advanced-Functions.pdf", 'total_pages': 14, 'page': 0, 'page_label': '1'}, page_content='Advanced Functions, \nGrade 12\nUniversity Preparation MHF4U\nThis course extends students’ experience with functions. Students will investigate the\nproperties of polynomial, rational, logarithmic, and trigonometric functions; develop\ntechniques for combining functions; broaden their understanding of rates of change; and\ndevelop facility in applying these concepts and skills. Students will also refine their use\nof the mathematical processes necessary for success in senior mathematics. This course \nis intended both for students taking the Calculus and Vectors course as a prerequisite 

In [187]:
## Vector Embedding And Vector Store
from langchain_community.vectorstores import Chroma
embeddings = OllamaEmbeddings(model = 'nomic-embed-text:latest')
db = Chroma.from_documents(documents,embeddings)

In [188]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
retriever  =  db.as_retriever()

# Strengths RAG

In [189]:
# Load the llm 
llm = OllamaLLM(model="gemma", temperature=0.4)

# Define prompt template
template = """
You are an assistant for question-answering tasks.
Use the provided context only to answer the following question:

<context>
{context}
</context>

Question: {input}

Here are the students quiz results summary that you can use to answer the question:
{quiz_res}



"""

# Create a prompt template
prompt = ChatPromptTemplate.from_template(template)

# Create a chain 
doc_chain = create_stuff_documents_chain(llm, prompt)
chain = create_retrieval_chain(retriever, doc_chain)

In [200]:
# User query 
response = chain.invoke({"input": "Give this student a detailed summary on all the things they did well in the quiz. Use the provided context to analyze what topics from the curriculum they did well in.", "quiz_res": res_strength})

# Get the Answer only
final_strength = response['answer']

In [201]:
print(final_strength)

The student demonstrated proficiency in several key areas covered in the curriculum:

**Mathematical Skills:**

* **Problem-solving:** The student tackled challenging questions involving average rate of change and function composition, showcasing their ability to apply mathematical concepts to solve complex problems.
* **Reasoning and Proving:** The student demonstrated proficiency in constructing mathematical arguments and assessing the reasonableness of results.
* **Representing Mathematical Concepts:** The student displayed an understanding of representing mathematical concepts in various forms, including words, numbers, graphs, and algebraically.

**Specific Topics:**

* **Exponential Equations:** Mastery of exponential equations was evident in their ability to solve related problems.
* **Radian Measure:** Understanding of radians was demonstrated through their accurate application in various contexts.
* **Trigonometric Identities:** The student's ability to utilize trigonometric i

## Weakness

In [192]:
# Load the llm 
llm = OllamaLLM(model="gemma", temperature=0.4)

# Define prompt template
template = """
You are an assistant for question-answering tasks.
Use the provided context only to answer the following question:

<context>
{context}
</context>

Question: {input}

Here are the students quiz results summary that you can use to answer the question:
{quiz_res}



"""

# Create a prompt template
prompt = ChatPromptTemplate.from_template(template)

# Create a chain 
doc_chain = create_stuff_documents_chain(llm, prompt)
chain = create_retrieval_chain(retriever, doc_chain)

In [193]:
# User query 
response = chain.invoke({"input": "Give this student a detailed learning path that strategises ways to improve their performance.", "quiz_res": res})

# Get the Answer only
final = response['answer']

In [194]:
print(final)

## Learning Path for Improving Student Performance

**Phase 1: Review and Reinforcement**

* Acknowledge the student's strengths in exponential equations, radians, trigonometric identities, polynomial roots, and function transformations.
* Provide targeted practice problems for the medium-difficulty questions on trigonometric identities and polynomial roots.
* Offer additional explanations and examples for the hard-difficulty questions on average rate of change and function composition.

**Phase 2: Targeted Intervention**

* **Average rate of change:**
    * Introduce the concept of derivatives and their connection to average rate of change.
    * Provide visual representations like graphs and tangent lines to understand the concept.
    * Practice applying derivative concepts to solve related problems.

* **Function composition:**
    * Review the concept of function composition and its properties.
    * Introduce advanced function composition techniques like inverse functions and com

# Translation

In [203]:
llm_lang = OllamaLLM(model = "stablelm2", temperature= 0.2)

# Define prompt template
template = """
You are an assistant for translating from English to **Mandarin.**

You are taking in a student detailed learning path but returning it in **Mandarin.**


Here is the student detailed learning path:
{}


## Output Structure:
- Make sure to keep the same format as the input.
- Only display output in **Mandarin**

"""

In [204]:
prompt_lang_strength = template.format(final_strength)

res_lang_strength = llm_lang.invoke(prompt_lang_strength)
print(res_lang_strength)

学生详细的学习路径：
学生表现了几个关键方面的优势：

**数学能力：**

* **解决复杂问题：** 学生在许多与平均速度的问题中展示了自用数学概念来解决复杂问题。
* **证明和推导：** 学生具有良好的构建数学命名式和评估结果的能力。他们可以将数学概念应用于解决复杂问题。
* **表示数学概念：** 学生明确了数学概念在不同形式中的理解，并且能够将其转换为字面、数字、图形和代数形式。例如，他们可以用“字”表示数学概念，还可以用“方程片段”的形式表示。
* **导数：** 学生明确地理解了对数学问题的关注点，并能够在不同背景下有效地应用这些知识。例如，在经典的代数中，他们可以准确地应用导数。
* **古振定式：** 学生了解到古振定式是如何在各种情况下使用的，并能够有效地解决相关问题。例如，在经典的代数中，他们可以准确地应用古振定式。
* **导数公式：** 学生了解到导数公式是如何在各种情况下使用的，并能够有效地解决相关问题。例如，在经典的代数中，他们可以准确地应用导数公式。
* **变换函数：** 学生了解到变换函数是如何在各种情况下使用的，并能够有效地解决相关问题。例如，在经典的代数中，他们可以准确地应用变换函数。
* **函数组合：** 存储学生在不同场景中展示了自己的理解和使用过程。例如，他们可以通过将两个或多个函数组成为一个新的函数来解决问题。
* **函数转换：** 学生了解到函数转换是如何在各种情况下使用的，并能够有效地解决相关问题。例如，在经典的代数中，他们可以准确地应用函数转换。

## 输出格式：
- 保持输入格式。
- 输出内容仅在**Mandarin**语言中显示。


In [205]:
prompt_lang = template.format(final)

res_lang = llm_lang.invoke(prompt_lang)
print(res_lang)

以下是学生详细的学习路径的中文版本：
### 学习路径优化：提升学生表现

**阶段1：审讯和重复性教学**

* 常识了解学生在古法方面的优势，例如古法、变换、线性代数、多项式根等。
* 提供针对中度问题的目标题材料，如三角函数和多项式根。同时，提供高度问题的例子，如平均速度和功能复分。
* 为难度较大的问题，例如线性变换、彩病和复数函数，提供解决相关问题的技巧，并给学生进行实际应用。

**阶段2..
...
### 学习路径优化：提升学生表现

* **平均速度（Average rate of change）：
     * 推引代数和其与平均速度的关系。同时，为图形和直线的视觉表示法进行了介绍。
     * 为难度较高的问题提供应用代数的实际应用，如解决相关问题的技巧。
     
* **函数组成（Function composition）：
     * 推引函数组成的概念和其属性。同时，介绍更高级的函数组成技巧，如反函数和复数函数。
     * 提供实际问题中使用函数组成的工具，以便在现实生活中应用数学思考。

**阶段3：扩展和增强**

* 许多学生兴趣更为挑战的数学概念，如几何、线性代数或高度 trigonometry。
* 从中提荐相关实际问题，例如计算干系率和函数复分。同时，建议在线资源，包括教程、挑战题解决工具以及在线社区，为学习进展提供支持。
* 从中推荐实际的数学在现实生活中的应用。

**额外策略：**

* 帮助学生跟踪他们的学术进展，以便追踪其学习路径的效果。
* 许多学生兴趣更强，那么他们可以与同学或教师进行共享学习活动。
* 提供针对学生的学习风格和速度的不同化妍杂料和资源。
* 从中提供适当的学习工具和资源，根据学生的进展进行调整。

**持续监视：**

* 在下一个测试题和assignment中追踪学生的学习路径的效果。
* 为了检查其学习路径的有效性，必...
### 学习路径优化：提升学生表现

**阶段1：审讯和重复性教学**

* 常识了解学生在古法方面的优势，例如古法、变换、线性代数、多项式根等。
* 提供针对中度问题的目标题材料，如三角函数和多项式根。同时，提供高度问题的例子，如平均速度和功能复分。
* 为难度较大的问题，例如线性变换、彩病和复数函数，提供解决相关问题的技巧，并给学生进行实际应用。

**

In [213]:
from langchain_openai import OpenAI

llm_video = LLM()


video_prompt = ''' 

You are an agent that takes in a student learning path and recommedns videos to watch based 
on the topics they need improved on.

Please return for a each topic a few video recommendations.

Each recommendation should be in the following format:

Video: (video title)
Link: (video link)

Here is the student learning path:

{}

## Output Structure:
- DO NOT DISPLAY PYTHONCODE
- DO NOT DISPLAY JSON
- DO NOT WRAP ANSWER IN (''' ''')


'''

In [214]:
video_prompt_final = video_prompt.format(final)

res_video = llm_video.llm.invoke(video_prompt_final)
print(res_video)

## Video Recommendations:

**Average Rate of Change:**

* **Video:** Introduction to Derivatives
* **Link:** https://www.khanacademy.org/math/calculus-1/derivative-introduction/average-rate-of-change/v/introduction-to-derivatives

* **Video:** Average Rate of Change and Secant Lines
* **Link:** https://www.youtube.com/watch?v=PwtNw5TLoiw

* **Video:** Applying Derivatives to Find Instantaneous Rate of Change
* **Link:** https://www.youtube.com/watch?v=8stueNPVl-I


**Function Composition:**

* **Video:** Function Composition - Explained!
* **Link:** https://www.youtube.com/watch?v=qxBmISCJSME

* **Video:** Inverse Functions
* **Link:** https://www.khanacademy.org/math/algebra2/x2ec2f6f830c9fb89:functions/x2ec2f6f830c9fb89:inverse-functions/v/introduction-to-inverse-functions

* **Video:** Composition of Functions - Real World Examples
* **Link:** https://www.youtube.com/watch?v=3UXLf-xK_U4 

